# Noteboook destinada al entrenamiento del modelo

## 1. Importar librerías y raw data
Todas las librerías (Pandas, NumPy, Seaborn...) y el propio código modular `src/`.

In [68]:
import sys
sys.path.insert(0, '../src/')
import pandas as pd
import importlib
import feature_engineering 
import preprocesing
importlib.reload(feature_engineering)
importlib.reload(preprocesing)
from feature_engineering import get_features, get_cat_num_features
from preprocesing import pipeline_preprocesamiento
from sklearn.model_selection import train_test_split

In [69]:
# Imporar dataset preprocesado
df = pd.read_csv('../data/processed/df_clean.csv')
target = 'minutos_visitados_mes'

### 2. Feature Engineering

Creación de nuevas features

In [73]:
df_new = get_features(df)
df_new.describe()

,aniomes,fecha,minutos_visitados_mes,cantidad_heladeras,venta_total_negocios_mes,cantidad_productos_total_negocios_vendidos_mes,cantidad_compras_total_negocios_mes,dias_entre_compras_total_negocios_mes,venta_negocio1_mes,venta_negocio2_mes,venta_negocio3_mes,venta_negocio4_mes,productos_por_compra,venta_promedio_x_compra,mes_sin,mes_cos,is_first_month,promedio_compras_cliente,promedio_productos_cliente,promedio_venta_total
count,196546.000000,196546,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.00000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000,196546.000000
mean,202441.928571,2024-10-16 01:42:51.428571648,57.082599,1.000356,2.729217,6.701831,11.667030,3.746224,1.370112,0.852977,0.461999,0.01449,0.432396,0.110919,0.097573,-0.097573,0.071429,11.667030,6.701831,2.729217
min,202404.000000,2024-04-01 00:00:00,0.000000,0.000000,-4.652640,0.000000,0.000000,0.000000,-3.405600,-8.165000,-10.035000,0.00000,0.000000,-0.369683,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,202407.000000,2024-07-01 00:00:00,14.916667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,-0.500000,-0.866025,0.000000,2.285714,1.571429,0.092841
50%,202410.500000,2024-10-16 12:00:00,30.075000,0.000000,0.210000,4.000000,5.000000,1.142857,0.000000,0.000000,0.000000,0.00000,0.500000,0.031676,0.250000,-0.250000,0.000000,6.571429,4.071429,0.446333
75%,202502.000000,2025-02-01 00:00:00,62.116667,1.000000,1.449540,9.000000,16.000000,3.500000,0.628380,0.442500,0.000000,0.00000,0.727273,0.101250,0.866025,0.500000,0.000000,15.428571,8.857143,1.791719
max,202505.000000,2025-05-01 00:00:00,1708.666667,78.000000,2069.747220,111.000000,591.000000,715.000000,1067.239360,2069.708840,678.780000,405.00000,1.000000,91.341000,1.000000,1.000000,1.000000,449.142857,78.214286,506.304340
std,45.574959,NaN,78.165055,1.707643,16.763103,9.173901,19.320242,11.747650,9.930544,9.932799,3.953884,1.23710,0.366893,0.537649,0.700344,0.700344,0.257540,15.877740,7.644019,11.651998


Separar variables numéricas de categóricas

In [74]:
cat_features, num_features = get_cat_num_features(df_new)

3 variables categóricas
19 variables numéricas
22 variables en total


In [75]:
# Elimino algunas features
cat_features = [f for f in cat_features if f not in ['cliente_id']]
num_features = [f for f in num_features if f not in [target, 'aniomes']]

In [76]:
X = df_new[cat_features + num_features]
y = df_new[target]

Separar conjunto de entrenamiento del testeo

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Preprocesar variables categóricas y numéricas
- A las variables numéricas las voy a normalizar con RobustScaler debido a la presencia de outliers en todas las features
- Las variables categóricas las codifico con One-Hot Encoding dado que son pocas categorías

In [96]:
preprocessor = pipeline_preprocesamiento(num_features, cat_features, 
                                               categorical_strategy='onehot', except_features=['mes_sin','mes_cos','is_first_month'])
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

### Clasificación de clientes - stand by despues vemos

### 3.  Modelo Baseline